### In this project, I'll use variational autoencoder to generate new molecules
### Even if the training is hard in this case, I've succeded to generate new molecules using the suitable probabilistic layer within the decoder.

In [12]:
import tensorflow as tf
#from tensorflow.keras.models import Sequential, Model
import tf_keras as tfk
import tensorflow_probability as tfp
import random
import numpy as np
tfd = tfp.distributions
tfpl = tfp.layers
tfb = tfp.bijectors

import tf_keras as tfk
tfkl = tfk.layers

from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split

### Preparation of data

In [13]:
DATA_PATH = "c:\\users\\talibit\\Drug_discovery_data\\alzheimersdata.txt"
data = open(DATA_PATH, 'r').read().splitlines()
new = []
count =0
for molecule in data:
    count+=1
    if 30 < len(molecule) < 50 and count < 300000:
        if ('T' not in molecule) and ('V' not in molecule) and ('g' not in molecule) and ('L' not in molecule) and ('8' not in molecule):
            new.append(molecule)

max_seq_len = len(max(new, key=len)) 

l = [['$']+ list(i) for i in new]
#l = [['$']+ [i] for i in new]
data = pad_sequences(l, maxlen=max_seq_len+1, padding="post", value="£",dtype=object) 
print(data.shape)
# so now each smile string is 50 character long element
#data = np.expand_dims(data, -1)
type(data)

(847, 50)


numpy.ndarray

In [14]:
tokenizer = Tokenizer(num_words=None, char_level=True, lower=False) 

tokenizer.fit_on_texts(data) 
new_data = tokenizer.texts_to_sequences(data) 
new_data = np.array(new_data)
new_data = np.expand_dims(new_data,-1)
new_data.shape

(847, 50, 1)

In [15]:
word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index)
print(f'number of words in word_index: {vocab_size}')
print(f'word_index: {word_index}')

number of words in word_index: 37
word_index: {'c': 1, '£': 2, 'C': 3, '(': 4, ')': 5, '1': 6, '2': 7, 'n': 8, 'O': 9, '3': 10, 'N': 11, '=': 12, '$': 13, '-': 14, '[': 15, ']': 16, '4': 17, 'H': 18, 'F': 19, '/': 20, 'o': 21, 'l': 22, 'S': 23, 'B': 24, 'r': 25, '@': 26, 's': 27, '#': 28, '\\': 29, '+': 30, '.': 31, '5': 32, 'I': 33, 'ï': 34, '»': 35, '¿': 36, 'P': 37}


In [16]:
seed = 1
np.random.seed(seed)
full_train, x_test = train_test_split(np.array(new_data), test_size=0.2, random_state=seed)

x_train, x_val = train_test_split(np.array(full_train), test_size=0.1, random_state=seed)

In [17]:
x_train.shape , x_val.shape, x_test.shape

((609, 50, 1), (68, 50, 1), (170, 50, 1))

### Let's build a VAE with a simple prior : a standard gaussian

In [18]:
## Now let's build a simple VAE that encode 50 characters logn smile string into 

latent_size = 10
Prior_distribution = tfd.MultivariateNormalDiag(loc=tf.zeros(latent_size, dtype=tf.float32),
                                        scale_diag=tf.ones(latent_size, dtype=tf.float32))


kl_regularizer= tfpl.KLDivergenceRegularizer(Prior_distribution,use_exact_kl = True)


In [19]:
encoder = tfk.Sequential([
        tfkl.LSTM(128, input_shape=(50,1), return_sequences=True),
        tfkl.Dropout(0.15),
        tfkl.MaxPooling1D(pool_size=2),
        tfkl.LSTM(256, return_sequences=True),
        tfkl.Dropout(0.05),
        tfkl.MaxPooling1D(pool_size=2),
        tfkl.LSTM(512, return_sequences=True),
        tfkl.Dropout(0.25),
        tfkl.Flatten(),
        tfkl.Dense(tfpl.MultivariateNormalTriL.params_size(latent_size)),
        tfpl.MultivariateNormalTriL(latent_size,
                                   activity_regularizer = kl_regularizer)


    ])
encoder.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_9 (LSTM)               (None, 50, 128)           66560     
                                                                 
 dropout_9 (Dropout)         (None, 50, 128)           0         
                                                                 
 max_pooling1d_2 (MaxPoolin  (None, 25, 128)           0         
 g1D)                                                            
                                                                 
 lstm_10 (LSTM)              (None, 25, 256)           394240    
                                                                 
 dropout_10 (Dropout)        (None, 25, 256)           0         
                                                                 
 max_pooling1d_3 (MaxPoolin  (None, 12, 256)           0         
 g1D)                                                 

In [20]:
decoder = tfk.Sequential([
    tfkl.Dense(10, activation = 'relu', input_shape =(latent_size,)),
    tfkl.Reshape((latent_size,1)),
    tfkl.LSTM(512, return_sequences=True),
    tfkl.Dropout(0.05),
    tfkl.UpSampling1D(size = 2),
    tfkl.LSTM(256, return_sequences=True),
    tfkl.Dropout(0.05),
    tfkl.UpSampling1D(size = 2),
    tfkl.LSTM(128, return_sequences=True),
    tfkl.Dropout(0.05),
    tfkl.Flatten(),
    tfkl.Dense(50 * (vocab_size+1)),  # Output 50 * 37 logits
    tfkl.Reshape((50, vocab_size+1)),  # Reshape to (50, 37)
    tfpl.DistributionLambda(lambda t: tfd.Independent(
        tfd.Categorical(logits=t), reinterpreted_batch_ndims=1))

])
decoder.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_5 (Dense)             (None, 10)                110       
                                                                 
 reshape_3 (Reshape)         (None, 10, 1)             0         
                                                                 
 lstm_12 (LSTM)              (None, 10, 512)           1052672   
                                                                 
 dropout_12 (Dropout)        (None, 10, 512)           0         
                                                                 
 up_sampling1d_4 (UpSamplin  (None, 20, 512)           0         
 g1D)                                                            
                                                                 
 lstm_13 (LSTM)              (None, 20, 256)           787456    
                                                      

In [21]:
vae = tfk.Model(inputs=encoder.inputs, outputs=decoder(encoder.outputs))

In [22]:
optimizer = tfk.optimizers.Adam(learning_rate=0.0005)
vae.compile(optimizer=optimizer, loss=lambda x, pred : -tf.reduce_mean(pred.log_prob(x)))

In [23]:
vae.fit(x = x_train, y = np.squeeze(x_train),validation_data = (x_val, np.squeeze(x_val)), epochs = 30)

Epoch 1/30

20/20 [==============================] - 14s 327ms/step - loss: 132.1225 - val_loss: 109.6865
Epoch 2/30
20/20 [==============================] - 5s 267ms/step - loss: 107.2357 - val_loss: 107.5117
Epoch 3/30
20/20 [==============================] - 5s 245ms/step - loss: 106.0026 - val_loss: 106.7814
Epoch 4/30
20/20 [==============================] - 5s 244ms/step - loss: 105.3973 - val_loss: 106.7591
Epoch 5/30
20/20 [==============================] - 5s 248ms/step - loss: 105.1384 - val_loss: 106.4057
Epoch 6/30
20/20 [==============================] - 5s 264ms/step - loss: 104.9971 - val_loss: 106.4109
Epoch 7/30
20/20 [==============================] - 5s 254ms/step - loss: 104.8721 - val_loss: 106.4357
Epoch 8/30
20/20 [==============================] - 5s 276ms/step - loss: 105.0916 - val_loss: 106.9826
Epoch 9/30
20/20 [==============================] - 6s 279ms/step - loss: 104.7470 - val_loss: 106.5633
Epoch 10/30
20/20 [==============================] - 5s 271ms/

In [24]:
# Evaluate the model on the test set
test_loss = vae.evaluate(x_test)
print("Test loss: {}".format(test_loss))

6/6 [==============================] - 2s 21ms/step - loss: 0.0093
Test loss: 0.009290237911045551


In [25]:
#Reconstruction of example

print(data.shape)

random_number = random.randint(0, 846)
print("the initial smile string is:",''.join(data[random_number,:].tolist()))

new_data = tokenizer.texts_to_sequences(data[random_number,:]) 
new_data = np.expand_dims(new_data,0)
new_data.shape
q = encoder(new_data)
q
p = decoder(q.mean())


sequence = p.sample().numpy()
tokenizer.sequences_to_texts(sequence)

(847, 50)
the initial smile string is: $COc1cccc(C2=C(c3cn(C)c4cc(F)ccc34)C(=O)NC2=O)c1££


['$ O O c ( c N 1 O c n c c c 2 - c 4 ( ( 3 c C 3 = ) c c ) n c ) c C n c c c C 3 £ £ £ 1 c c £ £ £ £']

In [29]:
# Generation
n_samples = 7000
samples = Prior_distribution.sample(n_samples)
output_distribution = decoder(samples)
smiles = tokenizer.sequences_to_texts(output_distribution.sample().numpy())

In [34]:
from rdkit import Chem
def checkSMILES(smiles):
	valid_smiles = []
	for smile in smiles:
		new = ''
		for char in smile:
			if char != '$' and char != '£':	
				new += char

		if Chem.MolFromSmiles(new, sanitize=False) is not None and len(new) > 15:
			valid_smiles.append(new)
			#print(new)

	return valid_smiles

In [35]:
checkSMILES(smiles)

[17:14:29] SMILES Parse Error: syntax error while parsing: C = C = N C C c c c ) c c O c O ( c n ) c H F ( C C O c c c n ) c C c c  C n  C 1       
[17:14:29] SMILES Parse Error: Failed parsing SMILES ' C = C = N C C c c c ) c c O c O ( c n ) c H F ( C C O c c c n ) c C c c  C n  C 1       ' for input: ' C = C = N C C c c c ) c c O c O ( c n ) c H F ( C C O c c c n ) c C c c  C n  C 1       '
[17:14:29] SMILES Parse Error: syntax error while parsing: N N c 1 c c O C 2 c ) c ( 2 3 n C c H C C c l ) 2 c ) c c ) 3 ) 4 2 c c  c c C C 1 1 ) o   C 
[17:14:29] SMILES Parse Error: Failed parsing SMILES ' N N c 1 c c O C 2 c ) c ( 2 3 n C c H C C c l ) 2 c ) c c ) 3 ) 4 2 c c  c c C C 1 1 ) o   C ' for input: ' N N c 1 c c O C 2 c ) c ( 2 3 n C c H C C c l ) 2 c ) c c ) 3 ) 4 2 c c  c c C C 1 1 ) o   C '
[17:14:29] SMILES Parse Error: syntax error while parsing: C 1 C 1 C N 2 C O c c ( c c c = S - O C ) c ( c C 2 ) n ( O c c c = C  2      )   C   
[17:14:29] SMILES Parse Error: Failed parsing S

['S C C 1 1 n c ( ( @ N / ( 1 C c c 1 2 2 / C 1 c ( c 2 ( - 2 c 3 4 4 c ( c c  2 3  1 ) 3 2   O ',
 'c O n c N c ) c ( O - c c / n C c 2 c c c N N ( c n 3 ( c O 2 ) c ) ( )  c O  1 c 1  c     ',
 'B C n 1 C C c ] c c 1 n ] O c c ) C c ( c c ) C c 3 ) ( - ( c c c = c c - n O N  ) 2  )  2   ',
 'c N = 1 1 / C 2 ( C 1 C ) [ n N c c H c C ) ) c l ) / ( c = c C n ] c ) = n B o )  1 )   n   ',
 'O C = ( ( / c ( ( c N [ ) ] C c ] ( 2 O 4 N ( c c O O c @ ] 1 ] ( = n c c N    2 1  C     1',
 'O C r c C ) c c c c C 1 H n = 3 3 n c c / F c c c C c 3 4 c c c ) c n c ) ( C 1 O c n  1 (  H  ',
 'N C O 1 1 ( 1 c ( O n c C c c 1 c ( ( C 3 c c C 1 3 c c = - 1 ( C 3 c ( C )   c c 2 c  2   F ',
 'P O c 1 c ( ( 2 c c n C o C C n - 2 c = n C c - c c c 3 c O ) = c C = ) 2 N c 3 C 1  c ) )    ',
 'O C O C ( c C N N = c n c C 3 C 3 c c c N c ( c c C = c ) = O 4 ) c c c n ] c 3  C 2   . c   ',
 'O C 1 ( n ( c c ( C - = ] 1 2 c n c c [ C c C c H = n 3 ) c c c 2 c ( c 2 C c ) N   c     ']

### the result was not satisfactory, we got just 10 corrected molecules out of 8000 samples from the prior